In [25]:
import os
import json
import zipfile

In [26]:
with open('kaggle.json', "r") as file:
    config = json.load(file)

os.environ['KAGGLE_USERNAME'] = config["username"]
os.environ['KAGGLE_KEY'] = config["key"]

import kaggle

In [ ]:
extract_path = "weightloss_data"
zip_path = "comprehensive-weight-change-prediction.zip"
os.makedirs(extract_path, exist_ok=True)


if not os.path.exists(f"{extract_path}\weight_change_dataset.csv"):
    !kaggle datasets download -d abdullah0a/comprehensive-weight-change-prediction
    with zipfile.ZipFile(zip_path, "r") as zip_ref:
        zip_ref.extractall(extract_path)
else:
    print("Data exists locally, will not re-download")


Data exists locally, will not re-download
